In [3]:
import os
import sys
import tensorflow as tf
import tensorflow_datasets as tfds

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [4]:
from data_loaders.example_data_loader import ExampleDatasetGenerator
from models.models import JukeboxModel
from models.preprocessing import AudioFeatureExtractor
from utils.dirs import create_dirs
from utils.config import process_config
from utils.utils import get_args

import datasets.canonne_duos.canonne_duos

ModuleNotFoundError: No module named 'tensorflow_datasets'